In [34]:
import numpy as np #maths library

# 8-Queens using Genetic Algorithm 

In [41]:
def initial_population(population_size):
    '''
    
    Generates an initial population of permutations of integers from 0 to 7.

    Parameters:
    - population_size (int): The size of the population to be generated.
 
    Returns:
    - numpy.ndarray: An array of shape (population_size, 8) containing the initial population
      of permutations of integers from 0 to 7
    
    '''
    population = []
    for _ in range(population_size):
        population.append(np.random.permutation(8))
    return np.array(population)

$fitness(solution) = -score(solution) $

In [42]:
def fitness(chromosome):
    '''
    Calculates the fitness score of a given chromosome in the context of the 8-Queens problem.

    The fitness score is determined by the number of conflicts present in the given chromosome. A conflict 
    occurs when two queens threaten each other, either by being in the same row, column, or diagonal. 
    The fitness score is negative, as the goal is to minimize the number of conflicts.

    Parameters:
    - chromosome (array-like): A permutation representing the placement of queens on the chessboard.
      Each element of the chromosome corresponds to a row, and the value at each element represents the
      column where the queen is placed.

    Returns:
    - int: The fitness score of the chromosome. A lower score indicates a better solution with fewer conflicts.
    '''
    score = 0
    for i in range(len(chromosome)):
        for j in range(len(chromosome)):
            if j == i:
                continue
            if abs(i - j) == abs(chromosome[i] - chromosome[j]) or chromosome[i] == chromosome[j]:
                score += 1
    return -score

In [43]:
def selection(population, fitness_vals):
    '''
    Selects individuals from a population based on their fitness values using a probabilistic selection method.

    Parameters:
        population (array-like): The population of individuals to select from.
        fitness_vals (array-like): The fitness values corresponding to each individual in the population.

    Returns:
        array-like: The selected individuals from the population.

    '''
    # Make a copy of fitness_vals to avoid modifying the original array
    probs = fitness_vals.copy()

    probs += abs(probs.min()) + 1

    # Normalize the fitness values to calculate selection probabilities
    probs = probs / probs.sum()

    # Number of individuals in the population
    n = len(population)

    # Generate an array of indices corresponding to individuals in the population
    indices = np.arange(n)

    # Sample individuals from the population based on their selection probabilities
    selected_indices = np.random.choice(indices, size=n, p=probs)
    selected_population = population[selected_indices]

    return selected_population


<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSSjPIuIsEsOJOnHWYa2RtTDc70T3gyu7-PEk6bkrsw&s" />

In [44]:
def crossover(parent1, parent2, threshold):
    '''
    Performs crossover between two parent chromosomes to produce offspring.

    Parameters:
        parent1 (array-like): The first parent chromosome.
        parent2 (array-like): The second parent chromosome.
        threshold (float): The probability threshold for crossover to occur. Should be in the range [0, 1].

    Returns:
        tuple: A tuple containing the two offspring chromosomes generated through crossover.

    Algorithm:
        1. Generate a random number between 0 and 1.
        2. If the random number is less than the threshold, perform crossover.
        3. Select a crossover point randomly between 1 and the length of the chromosomes.
        4. Create two offspring chromosomes by exchanging genetic material at the crossover point.
        5. If the random number is greater than or equal to the threshold, produce offspring identical to the parents.

    '''
    # Generate a random number between 0 and 1
    rnd = np.random.random()

    if rnd < threshold:
        # If random number is less than threshold, perform crossover
        crossover_point = np.random.randint(1, len(parent1))
        # Create two offspring chromosomes by exchanging genetic material at the crossover point
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    else:
        # If random number is greater than or equal to threshold, produce offspring identical to the parents
        child1 = parent1.copy()
        child2 = parent2.copy()

    return child1, child2


<img src = "https://raw.githubusercontent.com/KendallPark/genetic-algorithm/master/images/mutation.png"/>

In [45]:
def mutation(selected_population, threshold):
    '''
    Applies mutation to selected individuals from the population based on a given probability threshold.

    Parameters:
        selected_population (array-like): The selected individuals from the population.
        threshold (float): The probability threshold for mutation to occur. Should be in the range [0, 1].

    Returns:
        array-like: The mutated population.

    Algorithm:
        1. Generate a random number between 0 and 1.
        2. If the random number is less than the threshold, apply mutation.
        3. Select an individual from the population randomly.
        4. Mutate a randomly chosen gene in the selected individual.
    '''
    # Generate a random number between 0 and 1
    rnd = np.random.random()

    if rnd < threshold:
        # If random number is less than threshold, apply mutation
        index = np.random.randint(len(selected_population))
        # Select an individual from the population randomly
        individual = selected_population[index]
        # Mutate a randomly chosen gene in the selected individual
        gene_index = np.random.randint(len(individual))
        individual[gene_index] = np.random.randint(8)

    return selected_population


In [49]:
def crossover_mutation(selected_population, threshold_cross, threshold_mut):
    '''
    Applies crossover and mutation to pairs of selected individuals from the population.

    Parameters:
        selected_population (array-like): The selected individuals from the population.
        threshold_cross (float): The probability threshold for crossover to occur. Should be in the range [0, 1].
        threshold_mut (float): The probability threshold for mutation to occur. Should be in the range [0, 1].

    Returns:
        array-like: The new population resulting from crossover and mutation.

    Algorithm:
        1. Iterate through pairs of selected individuals from the population.
        2. For each pair, perform crossover to produce two offspring.
        3. Apply mutation to each offspring based on the mutation threshold.
        4. Add the mutated offspring to the new population.
        5. Repeat until all selected individuals are processed.

    '''
    population = []

    # Iterate through pairs of selected individuals from the population
    for i in range(len(selected_population) // 2):
        # Perform crossover to produce two offspring
        child1, child2 = crossover(selected_population[2 * i], selected_population[2 * i + 1], threshold_cross)
        # Apply mutation to each offspring based on the mutation threshold
        child1 = mutation(child1, threshold_mut)
        child2 = mutation(child2, threshold_mut)
        # Add the mutated offspring to the new population
        population.append(child1)
        population.append(child2)

    return np.array(population)


In [55]:
def display_solution(solution):
    '''
    Displays the solution of the Eight Queens problem on an 8x8 chessboard grid.

    Parameters:
        solution (array-like): The solution representing the placement of queens on the chessboard.

    '''
    n = len(solution)
    for row in range(n):
        line = ""
        for col in range(n):
            if solution[row] == col:
                line += " Q " 
            else:
                line += " . " 
        print(line)


In [56]:
def eight_queens(population_size, max_generation, threshold_cross=0.5, threshold_mut=0.05):
    '''
    Solves the Eight Queens problem using a genetic algorithm.

    Parameters:
        population_size (int): The size of the population.
        max_generation (int): The maximum number of generations to run the genetic algorithm.
        threshold_cross (float, optional): The probability threshold for crossover to occur. Defaults to 0.5.
        threshold_mut (float, optional): The probability threshold for mutation to occur. Defaults to 0.05.

    Algorithm:
        1. Initialize a population of random chromosomes.
        2. Repeat for a maximum number of generations:
            a. Evaluate the fitness of each chromosome in the population.
            b. Select the best individuals based on their fitness values.
            c. Perform crossover and mutation to generate a new population.
            d. Repeat until the best solution is found or the maximum number of generations is reached.

    '''
    population = initial_population(population_size)
    best_fitness_overall = None
    
    # Iterate through generations
    for i_gen in range(max_generation):
        # Evaluate the fitness of each chromosome in the population
        fitness_vals = np.array([fitness(chromosome) for chromosome in population])
        best_index = np.argmax(fitness_vals)
        best_fitness = fitness_vals[best_index]
        
        # Update the best overall fitness and solution found so far
        if best_fitness_overall is None or best_fitness > best_fitness_overall:
            best_fitness_overall = best_fitness
            best_solution = population[best_index]
        
        # Print the current generation and best fitness
        print(f'Generation = {i_gen} Fitness = {-1*best_fitness_overall}', end='\r')
        
        # Check if the solution is found
        if best_fitness == 0:
            print('\nSolution found')
            break
        
        # Select the best individuals based on their fitness values
        selected_population = selection(population, fitness_vals)
        
        # Perform crossover and mutation to generate a new population
        population = crossover_mutation(selected_population, threshold_cross, threshold_mut)
        
    print(f"Best Solution: {best_solution}\n")
    display_solution(best_solution)


In [57]:
eight_queens(100,5000, 0.7, 0.3)

Generation = 34 Fitness = 0
Solution found
Best Solution: [4 1 7 0 3 6 2 5]

 .  .  .  .  Q  .  .  . 
 .  Q  .  .  .  .  .  . 
 .  .  .  .  .  .  .  Q 
 Q  .  .  .  .  .  .  . 
 .  .  .  Q  .  .  .  . 
 .  .  .  .  .  .  Q  . 
 .  .  Q  .  .  .  .  . 
 .  .  .  .  .  Q  .  . 
